In [4]:
threshold = 40
video_feed = 0 # change this with path of video (currently set to WebCam)


import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

face_cascade = cv2.CascadeClassifier('Downloads/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('Downloads/haarcascade_eye_tree_eyeglasses.xml')
cap = cv2.VideoCapture(video_feed)

while cap.isOpened():
    
    _ , img = cap.read()
    stage = "No Eyes"
    
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img.flags.writeable = False
    results = pose.process(img)
    img.flags.writeable = True
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
    try:
        landmarks = results.pose_landmarks.landmark
        nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y]
#         print(nose)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        eyes = eye_cascade.detectMultiScale(gray)
#         print(eyes)
        
        for ex,ey,ew,eh in eyes:
            if((ex/100-nose[0])**2+(ey/100-nose[1])**2 > threshold):
                continue
            cv2.rectangle(img, (ex,ey), (ex+ew,ey+eh),(0,255,0), 5)
            stage = "Eye Detected"
    
    except:
        pass
    
    cv2.rectangle(img, (0,0), (400,80), (245,117,16), -1)
         
    cv2.putText(img, 'STATUS', (100,12), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(img, stage, 
                (50,50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('Eye Detector',img)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
                      
cap.release()
cv2.destroyAllWindows()